In [1]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

os.environ["SPARK_HOME"] = "/usr/local/spark"
os.environ["PYSPARK_PYTHON"] = "/home/pigidser/anaconda3/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "pyspark-shell"

spark = SparkSession.builder.master("local").appName("spark_test").getOrCreate()

In [2]:
# create two dataframes
person = spark.createDataFrame([
(0, "Bill Chambers", 0, [100]),
(1, "Matei Zaharia", 1, [500, 250, 100]),
(2, "Michael Armbrust", 1, [250, 100])]) \
.toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
(0, "Masters", "School of Information", "UC Berkeley"),
(2, "Masters", "EECS", "UC Berkeley"),
(1, "Ph.D.", "EECS", "UC Berkeley")]) \
.toDF("id", "degree", "department", "school")

In [3]:
person.show()
graduateProgram.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [4]:
joinExpression = person["graduate_program"] == graduateProgram['id']

In [5]:
# Inner

joinType = "inner"
person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [6]:
# Outer
# появляются строки из правого фрейма, для которых нет ключей в левом

joinType = "outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [7]:
# Left outer
# поменяли местами датафреймы, теперь все строки из левого датафрейма показаны, не для всех нашлись ключи в правом

joinType = "left_outer"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|   0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|   2|Michael Armbrust|               1|     [250, 100]|
|  2|Masters|                EECS|UC Berkeley|null|            null|            null|           null|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+



In [8]:
# Right outer
# ключи из правого датафрейма, не для всех нашлось соответствие в левом

joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|   2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|null|            null|            null|           null|  2|Masters|                EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



In [9]:
# Left semi
# опять поменяли местами датафреймы. Остались только те программы (левый датафрейм),
# для которых были соответствия в правом. Правого датафрейма нет (т.е. чисто работа со строками одного датафрейма).

joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [ ]:
# Left anti
# опять поменяли местами датафреймы. Остались только те программы (левый датафрейм),
# для которых не было соответствия в правом. Правого датафрейма нет (т.е. чисто работа со строками одного датафрейма).

joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()
+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+

In [10]:
# Union
# Просто добавим датафрейм в "хвост" себе же

graduateProgram.union(graduateProgram).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [11]:
# Поменяем местами колонки в добавляемом датафрейме - увидим, какая "каша" получилась.

graduateProgram.union(graduateProgram.select("degree","id","department","school")).show()

+-------+-------+--------------------+-----------+
|     id| degree|          department|     school|
+-------+-------+--------------------+-----------+
|      0|Masters|School of Informa...|UC Berkeley|
|      2|Masters|                EECS|UC Berkeley|
|      1|  Ph.D.|                EECS|UC Berkeley|
|Masters|      0|School of Informa...|UC Berkeley|
|Masters|      2|                EECS|UC Berkeley|
|  Ph.D.|      1|                EECS|UC Berkeley|
+-------+-------+--------------------+-----------+



In [12]:
# Если воспользоваться методом unionByName, то все будет корректно даже с переставленными колонками.

graduateProgram.unionByName(graduateProgram.select("degree","id","department","school")).show()


+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [13]:
spark.stop()

## SQL vs Spark

In [ ]:
"""
select con.contract_id as contract_id, con.date_insert as date_insert, con.date_sign as date_sign,
    con.begin_date as begin_date, con.end_date as end_date, con.product_id as product_id,
    con.channel_sale_id as channel_sale_id, con.contract_prev_id as contract_prev_id,
    con.contract_option_id as contract_option_id, con.owner_subject_id as owner_subject_id,
    con.contract_type_id as contract_type_id, cst.contract_status_type_id as status_type_id,
    sbj.subject_type_id as subject_type_id, sbj.subject_name as subject_name, pp.birth_date as birth_date,
    uv.hid_party as hid_party
from kasko.contract con
    join kasko.contract_status cst on cst.contract_status_id = con.contract_status_id
    join kasko.subject sbj on sbj.subject_id = con.owner_subject_id
    left join kasko.physical_person pp on pp.subject_id = con.owner_subject_id
    left join kasko.united_view uv on uv.source_id = sbj.subject_id and uv.source_name = 'UNI'
where department_id=3075
"""

In [ ]:
# таблица contract
qCon = """
select
    contract_id as con_contract_id,
    date_insert as con_date_insert,
    date_sign as con_date_sign,
    begin_date as con_begin_date,
    end_date as con_end_date,
    product_id as con_product_id,
    channel_sale_id as con_channel_sale_id,
    contract_prev_id as con_contract_prev_id,
    contract_option_id as con_contract_option_id,
    owner_subject_id as con_owner_subject_id,
    contract_status_id as con_contract_status_id,
    contract_type_id as con_contract_type_id
from
    kasko.contract
where
    department_id = 3075
"""
dfCon = sp.sql(qCon)

# таблица contract_status
qCStat = """
select 
    contract_status_id as cst_status_id,
    contract_status_type_id as cst_status_type_id
from
    kasko.contract_status
"""
dfCStat = sp.sql(qCStat)

# таблица subject
qSubj = """
select
    subject_id as sbj_subject_id,
    subject_type_id as sbj_subject_type_id,
    subject_name as sbj_subject_name
from
    kasko.subject
"""
dfSubj = sp.sql(qSubj)

# таблица physical_person
qPPers = """
select
    subject_id as pp_subject_id,
    birth_date as pp_birth_date
from
    kasko.physical_person
"""
dfPPers = sp.sql(qPPers)

# таблица united_view
qUView = """
select
    source_id as uv_source_id,
    hid_party as uv_hid_party
from
    kasko.united_view
where
    source_name = 'UNI'
"""
dfUView = sp.sql(qUView)

# Все эти датафреймы нам могут понадобиться многократно (в нашем конвейере преобразований данных),
# можно включить в них побольше колонок (на все случаи жизни).

# Далее формулируем правила связывания датафреймов, это всего лишь выражения, которые показывают
# как связаны между собой таблицы. Эти знания универсальны, могут быть унаследованы из схемы
# реляционной базы данных.

con_stat = f.col("cst_status_id")==f.col("con_contract_status_id")
con_subj_own = f.col("con_owner_subject_id")==f.col("sbj_subject_id")
con_ppers_own = f.col("con_owner_subject_id")==f.col("pp_subject_id")
subj_uview = f.col("sbj_subject_id")==f.col("uv_source_id")

# И, наконец, формулируем наш "многостраничный" запрос

resDf = dfCon.join(dfSubj,con_subj_own,"inner")\
    .join(dfCStat,con_stat, "inner")\
    .join(dfPPers,con_ppers_own, "left")\
    .join(dfUView,subj_uview,"left")